In [1]:
import threading
from tkinter import *
from tkinter.ttk import Combobox, Style
from PIL import Image, ImageTk, ImageEnhance
import requests
from bs4 import BeautifulSoup
import re

class CricketScore:
    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title('LIVE CRICKET SCORE')
        self.rootWindow.geometry('1000x500+100+100')

        # Custom style
        style = Style()
        style.configure('TButton', font=('Calibri', 12), relief='raised', padding=5)
        style.configure('TLabel', font=('Calibri', 12), background='#ADD8E6', padding=5)

        try:
            # Load the image
            original_image = Image.open(r"C:\Users\PMYLS\Desktop\openendedSDA\IK.webp").resize((1000, 500))
            self.bg_image = ImageTk.PhotoImage(original_image)
            self.bg_label = Label(self.rootWindow, image=self.bg_image)
            self.bg_label.place(x=0, y=0)
        except FileNotFoundError:
            print("Error: Background image not found. Please provide a valid path.")
        except Exception as e:
            print("Error loading image:", e)

        # Header Label
        self.label = Label(
            self.rootWindow,
            text='Live Matches',
            font=("Calibri", 30, "bold"),
            bg="White",
            fg="Green",
            relief="groove",
            borderwidth=3
        )
        self.label.place(x=380, y=50)

        # Dropdown and Buttons
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = list(self.matches.keys())
        self.cb = Combobox(
            self.rootWindow,
            values=self.data,
            textvariable=self.var,
            width=50,
            state="readonly",
            font=("Calibri", 12)
        )
        self.cb.place(x=300, y=150)
        self.cb.set("Select a Match")

        self.b1 = Button(
            self.rootWindow,
            text="Check Score",
            font=("Calibri", 12, "bold"),
            bg="LightGreen",
            fg="Black",
            relief="raised",
            borderwidth=3,
            command=self.show_match_details
        )
        self.b1.place(x=450, y=200)

        self.refresh_button = Button(
            self.rootWindow,
            text="Refresh Matches",
            font=("Calibri", 12, "bold"),
            bg="SkyBlue",
            fg="Black",
            relief="raised",
            borderwidth=3,
            command=self.refresh_matches
        )
        self.refresh_button.place(x=430, y=250)

    def refresh_matches(self):
        self.matches = self.match_details()
        self.data = list(self.matches.keys())
        self.cb['values'] = self.data  # Update combobox values
        self.cb.set("Select a Match")  # Reset the combobox selection
          
    def show_match_details(self):
        selected_match = self.var.get()
        if not selected_match or selected_match not in self.matches:  # Check for empty selection
            print("Invalid match selection")
            return

        for widget in self.rootWindow.winfo_children():
            if isinstance(widget, Frame):
                widget.destroy()

        self.frame1 = Frame(self.rootWindow, bg="#FFFFFF", relief="raised", borderwidth=4)
        self.frame1.place(x=150, y=300, width=700, height=150)

        match_data = self.matches[selected_match]
        Label(
            self.frame1,
            text=f"{selected_match} - {match_data['match_header']}",
            font=("Calibri", 15, "bold"),
            bg="#FFFFFF",
            fg="DarkBlue",
            anchor="w",
            wraplength=680
        ).place(x=10, y=10)
        Label(
            self.frame1,
            text=f"Score Details: {match_data['score_card']}",
            font=("Calibri", 12),
            bg="#FFFFFF",
            fg="Black",
            anchor="w",
            wraplength=680
        ).place(x=10, y=50)
        Label(
            self.frame1,
            text=f"Summary: {match_data['summary']}",
            font=("Calibri", 12),
            bg="#FFFFFF",
            fg="Black",
            anchor="w",
            wraplength=680
        ).place(x=10, y=90)

    def match_details(self):
        details = self.scrap()
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_header(detail)
            if summary:
                match_header = self.match_header(detail)
                teams = self.teams_name(detail)
                score_card = self.team_score(detail)
                if score_card and len(score_card) == 2:  # Check if score_card has both teams
                    live_team_details['summary'] = summary
                    live_team_details['match_header'] = match_header
                    live_team_details['score_card'] = score_card[0] + " :: " + score_card[1]
                    live_match[f"{teams[0]} vs {teams[1]}"] = live_team_details
        return live_match

    def team_score(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")
        if team1_details != "Data not available" and team2_details != "Data not available":
            t.append(team1_details)
            t.append(team2_details)
        return t

    def teams_name(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")

        if team1_details != "Data not available" and team2_details != "Data not available":
            team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
            team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)
            t.append(team1_details[:team1_index].strip())
            t.append(team2_details[:team2_index].strip())

        return t

    def match_header(self, detail):
        match_header = detail.find("div", class_="cb-mtch-crd-hdr")
        return match_header.text.strip() if match_header else "No Match Header"

    def safe_find(self, detail, tag, class_):
        element = detail.find(tag, class_=class_)
        return element.text.strip() if element else "Data not available"

    def scrap(self):
        try:
            URL = "https://www.cricbuzz.com/"
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find(id="match_menu_container")
            if results:
                return results.find_all("li", class_="cb-match-card")
            else:
                print("Could not find match container on the page.")
                return []
        except requests.exceptions.RequestException as e:
            print(f"Error during web request: {e}")
            return []
        except Exception as e:
            print("Error scraping data:", e)
            return []

def main():
    rootWindow = Tk()
    CricketScore(rootWindow)
    rootWindow.mainloop()

if __name__ == '__main__':
    main()


Error: Background image not found. Please provide a valid path.
